*JSC270, Winter 2020 - Prof. Chevalier*

# <center>  Assignment 2 - An Analysis on Yelp Businesses and Reviews </center>

### <center> Zhiyuan Lu, Feburary 9 </center>
<center>  Explore information and reviews about all businesses and GTA businesses from the Yelp Dataset</center>

***

## Part 1. Introduction
For this assigment, I used data from Yelp Data Challenge to explore information about businesses and the reviews they received. This information would be useful for both consumers and business owners.

### 1.1 Permissions  
We are allowed to use the Yelp data to create an academic project for a course. We cannot use the data for commercial purposes and any other purposes, and we cannot create our own business listings that may compete with Yelp. We are not allowed to use the data in a way that damages Yelp's image and profits, either. Finally, we cannot share or redistribute the data.

### 1.2 Structure of dataset  
There are six files in the Yelp dataset:    

**business.json:**
This file contains the information about every business. For each business, the following information is recorded:  
Basic information: `business id`, `name`  
Location: `address`, `city`, `state`, `postal code`, `latitude`, `longitude`  
`stars`: number of stars out of 5; an indication of the overall quality of the business.    
`review count`: number of reviews received.  
`is open`: a variable that takes the value of 0 or 1. 0 means the business is not open, and 1 means the business is open.  
`attributes`: a nested dictionary of attributes (e.g. 'GoodForKids', 'RestaurantsReservations') that each takes the value of True or False.  
`categories`: a list of categories of food or service provided.  
`hours`: a dictionary where the keys are day of the week, and the values are opening hours on each day.  

**checkin.json:**
This file contains the check-in log every restaurant. For each restaurant, two variables are recorded:   
`business id`: the unique id of every restaurant.      
`date`: a list of check-in time of a restaurant.

**photo.json:**
This file contains the information of every photo. For each photo, four variables are recorded:  
`caption`: description of the photo. The caption is missing for some photos.     
`photo id`: the unique id of every photo.  
`business id`: the id of the business where the photo is taken.  
`label`: the content of the photo that takes a value of either "food", "inside", or "outside".

**review.json:**
This file contains the information of every review. For each review, the following information is recorded:  
`review id`: the unique id of each review.     
`user id`: the id of the user who wrote this review.  
`business id`: the id of the business which the review is written for.  
`stars`: number of stars out of 5; an indication of the overall quality of the restaurant.    
`useful`: number of users who thought this review is useful.  
`funny`: number of users who thought this review is funny.  
`cool`: number of users who thought this review is cool.  
`text`: the content of the review.

**tip.json:**
This file contains a short tip written by users. For each tip, five variables are recorded:  
`text`: text of the tip.     
`date`: when the tip was written, formatted like YYYY-MM-DD.  
`business id`: the id of the restaurant where the tip is for.    
`user_id`: user who wrote this tip.   
`compliment count`: how many compliments it has.

**user.json:**
This file contains the information of every user. For each user, the following information is recorded:  
`user id`: the unique id of the user.  
`name`: name of user.  
`review count`: number of reviews this user has written.  
`yelping since`: time when registered.  
`useful`/`funny`/`cool`: number of "useful/funny/cool" votes this user received.  
`elite`: a list of the years when the user reaches elite status.  
`friends`: a list of user ids of the user's friends.  
`fans`: number of fans the user has.  
`average stars`: average rating of all reviews.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import json  
import pandas as pd  
from pandas.io.json import json_normalize  

import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

import plotly.express as px
import plotly.graph_objects as go

from IPython.core.display import HTML, Latex
from fuzzywuzzy import fuzz

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter

In [3]:
import geopy.distance

## Data Cleaning

We will use data of all businesses from business.json. First, I will read the json data by each row, and normalize to expand the nested dictionary of the `attributes`.

In [4]:
# Load data from json
for line in open('business_s.json', 'r', encoding='utf8'):
    # read each business line by line
    data = json.loads(line)
    # normalize to expand attributeds
    curr = json_normalize(data, max_level=1)
    frames.append(curr)
    
    results = pd.concat(frames, sort=False)

# save a csv file for later use    
# results.to_csv('business_expanded.csv', index=False)

NameError: name 'frames' is not defined

In [5]:
# if the above cell takes too long to run, run this cell instead
business = pd.read_csv('business_expanded.csv')

The `city` column in the business dataset has the following problems that need to be addressed in data cleaning:
1. The city information in one row is missing.
2. Extra whitespace between words.  
3. Some entries include the state information.  
4. Some entries use all upperclass or lowerclass characters.  
5. Special characters are not correctly shown, for example, French letters in 'Montreal'.  
6. Spelling mistakes.  

First, I will fix problem 1 by finding the city with the latitude and longitude of this business.

In [7]:
no_city = business[business["city"].isnull()]

In [8]:
geolocator = Nominatim(user_agent="my-application")
place = str(no_city.iloc[0].latitude) + ", " + str(no_city.iloc[0].longitude)

# get address of the business by reversing from (lat, lon)
location = geolocator.reverse(place)
print(location.address)

East Warner Road, Mesa, Maricopa County, Arizona, 85212, United States of America


In [9]:
business = business.replace({'city': {business.iloc[34070]['city']: 'Maricopa'}})

Problem 2-4 can be easily fixed with python functions. 

In [10]:
def clean_city(string):
    '''
    Clean the city name. The input city is a string of the city name.
    This function returns the cleaned name of the city.
    '''
    # Split by ',' and only keep the first part, to remove state information after ','
    result = string.strip().split(',')
    result = result[0]
    # Clean the string word by word
    result = result.split(' ')
    
    name = ''
    for i in range(len(result)):
        # remove extra whitespace between words
        if result[i] != '':
            # capitalize each word
            name = name + result[i].capitalize() + ' '
    
    # remove whitespace
    name = name.strip()
    return name

In [11]:
business['city'] = business.apply(lambda x: clean_city(x['city']),axis=1)

For problem 5, most of cities with French letters in city name are in Quebec, Canada. I only fixed the name for Montreal, because many businesses in Montreal are recorded in the dataset. However, I left the other city names unchanged, because it would be too time-consuming to manually search and fix the name for every city, and I will not use information from these cities in later analysis.

In [12]:
# Replace incorrect spellings of Montreal
business = business.replace({'city': '^Mont.*$'}, {'city': 'Montreal'}, regex=True)

For problem 6, I matched the city and state using Nominatim with Geopy. If the latitude and longitude of a city can be found, I would consider the city name is correct. For city names that cannot be matched, I used fuzzywuzzy to correct the spelling mistakes. After fixing problem 1-4, there are 1103 distinct (city, state) pairs in the original business dataset, 1008 cities are matched using Nominatim, and 95 are unmatched. After fuzz matching, 44 cities are still unmatched, and I excluded these 44 cities in my analysis.

In [13]:
# get latitude and longitude of city by calculating the mean latitude and longitude of all businesses in this city
# get mean stars and review counts of businesses in a city
city_loc = business.groupby(['city', 'state'])['latitude', 'longitude', 'stars', 'review_count'].mean()\
            .reset_index()

# get total number of businesses of a city
cities = business.groupby(['city', 'state'])['city'].count().reset_index(name='count')

# combine to a single dataframe                                           
cities = pd.merge(city_loc, cities, on=['city', 'state']).sort_values(['count'], ascending=False)

In [ ]:
rows= []
not_found = []

for i in range(len(cities)):
    other_cols = cities.iloc[i]
    other_cols = other_cols.to_dict()
    
    geolocator = Nominatim(user_agent="my-application")
    # match by city and state
    place = cities.iloc[i]['city'] + ',' + cities.iloc[i]['state']
    
    try:
        location = geolocator.geocode(place, timeout=10)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s"%(place)) 
    
    if location is not None: # city is matched, meaning the name is correct
        rows.append(other_cols)
    else: # incorrect city name
        not_found.append(other_cols)

# create a new df
city_df = pd.DataFrame(rows)
not_found = pd.DataFrame(not_found)

In [ ]:
matched = {}
for idx, miss in not_found.iterrows():
    max_score = 0
    max_index = 0
    
    # Compare the similarity of the incorrect city name to each correct city name 
    for i, exist in city_df[['city', 'state', 'count']].iterrows():
        score = fuzz.ratio(miss['city'], exist['city'])
        
        if score > 80 and score > max_score and miss['state'] == exist['state']:
            max_score = score
            max_index = i
            not_found.at[idx, 'city'] = exist['city']    
    
    # warn if the city cannot be matched
    if max_score > 80:
        matched[miss['city']] = not_found.at[idx, 'city']
        city_df.at[i, 'count'] += not_found.at[idx, 'count']

I replaced the misspelled city names with correct names, and saved this dataframe to csv for later use.

In [ ]:
business = business.replace({'city':matched})

In [ ]:
business.to_csv("business_cleaned.csv", index=False)

In [ ]:
# city_df.to_csv('cities.csv', index=False)

## Part 2 Investigating All Businesses
### 2.1 What cities does this dataset encompass?

We can plot a map of all the cities encompassed in the dataset. The map shows obvious clustering around cities with most businesses recorded, for example, there are clusters around Las Vegas in Nevada, Phoenix in Arizona, Charlotte in North Carolina, Pittsburgh in Pennsyvalnia, Toronto in Ontario, and Montreal in Quebec. This is because around those major cities, many businesses are recorded under the name of smaller neighbouring cities or counties.

In [ ]:
fig = px.scatter_mapbox(city_df, lat="latitude", lon="longitude", hover_name="city", hover_data=["count"],
                        opacity=0.2, zoom=3, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<center>
    <img src="map1.png"></img>
</center>

#### Limitations
1. The definition of "city" in this data set is ambiguous. Should a county or district under a city be considered separately or included in the main city? In this dataset we see both happen.

### 2.2 What are the most frequent business categories overall? 

First, I splitted the list of categories to count each category separately. For this analysis, I only considered opening businesses.

In [ ]:
def explode_categories(df, top):
    """
    Explode the list of categories information of dataframe df,
    group by a list of columns col,
    and return a dataframe containing the most popular categories by count.
    """
    df_explode = df.assign(categories = df.categories.str.split(', ')).explode('categories')
    popular = df_explode.groupby(['categories'])['categories'].count().reset_index(name='count') \
                             .sort_values(['count'], ascending=False).head(top)
    
    return popular

In [ ]:
popular = explode_categories(business, 15)

# select open businesses only
business_open = business[business['is_open']==1]
popular_open = explode_categories(business_open, 15)

In [ ]:
def plot_stacked_h(df1, df2, color, title):
    """
    Create a stacked bar plot for number of open and total businesses.
    """
    # Initialize the matplotlib figure
    f, ax = plt.subplots(figsize=(16, 8))

    plt.title(title, fontdict = {'fontsize' : 16})

    # Plot total businesses
    sns.set_color_codes("pastel")
    sns.barplot(x = 'count',
                y = 'categories',
                color = color,
                label = 'Total',
                data = df1)

    # Plot open businesses
    sns.set_color_codes("muted")
    sns.barplot(x = 'count',
                y = 'categories',
                color = color,
                label = 'Open',
                data = df2)

    # Add a legend and informative axis label
    ax.legend(ncol=2, loc="lower right", frameon=True)
    ax.set(ylabel='',
           xlabel="Number of businesses")
    sns.despine(left=True, bottom=True)

    plt.show()

In [ ]:
plot_stacked_h(popular, popular_open, 
               'b', 
               "Number of total and open businesses in the top 15 most popular categories")

The most popular categories among all businesses are Restaurants, Shopping, Food, Home Services, and Beauty & Spas. While Restaurants is the most popular category, it also has the highest number of closed businesses.  
Next, I decided to look further into food-related businesses and find the most popular sub-categories in Restaurants and Food.

In [ ]:
# select food-related businesses
food = business[business['categories'].str.contains(
                         'Restaurants|Food', case=False, na=False)]

popular_food = explode_categories(food, 17)
# remove rows of 'Restaurants' and 'food'
popular_food = popular_food[2:]

# select open restaurants only
food_open = food[food['is_open']==1]
popular_food_open = explode_categories(food_open, 17)
popular_food_open = popular_food_open[2:]

In [ ]:
plot_stacked_h(popular_food, popular_food_open, 
               'm', 
               "Number of total and open restaurants in the top 15 most popular categories of restaurants")

Nightlife, Fast Food, and Bars are the most popular. We also see American(Traditional) and American(New) among the top 15 categories, which is expected as the dataset contains businesses in North America, with a lot of them in the United States.

#### Limitations
1. I did not clean the category names using fuzzywuzzy. There could be misspelled names or abbrevations.
2. Many of the businesses in the original dataset do not have complete information about all categories. For example, some may be only labeled with 'Restaurant', without any of the sub-categories.

### 2.3 What types of establishments tend to have bike parking?

In the dataset of 192609 businesses, 66331 of them (34.4%) have bike parking, 18880 (9.8%) don't have bike parking, and the information is missing for 107398 businesses (55.8%). 

First, I choose to zoom in on the top 10 cities with most businesses recorded, and calculate the ratio of businesses with bike parking in each of these 10 cities. I then calculated the ratio difference of each city with the average business-with-bike-parking ratio of all businesses in the dataset.

In [ ]:
# find business whose bike parking attribute is true
bike_parking = business[business['attributes.BikeParking'] == 'True']

# find business whose bike parking attribute is false
no_bike_parking = business[business['attributes.BikeParking'] == 'False']

# find business does not have bike parking attribute
missing_info = business[business['attributes.BikeParking'].isna()]\
            .append(business[business['attributes.BikeParking'] == 'None'])

In [ ]:
# select top 10 cities with most businesses
top_cities = business.groupby(['city'])['city'].count()\
            .reset_index(name='count').sort_values('count', ascending=False).head(10)

# create a list of city names of the 10 cities
top_cities = top_cities['city'].to_list()

# select businesses in these 10 cities
major_city_business = business.loc[business['city'].isin(top_cities)]

In [ ]:
ratio = []

for city in top_cities:
    # select businesses in this city
    city_business = major_city_business.loc[major_city_business['city'] == city]
    # select businesses with bike
    with_bike = city_business[city_business['attributes.BikeParking'] == 'True']
    
    # calculate ratio difference between this city and the average among all cities
    city_ratio = len(with_bike) / len(city_business) - len(bike_parking) / len(business)
    ratio.append(city_ratio)

In [ ]:
bike_parking_ratio = pd.DataFrame({'city':top_cities,
                     'ratio difference': ratio}).sort_values('ratio difference', ascending=False)

# create a categorical variable to plot different colors for positive and negative values
bike_parking_ratio['positive'] = bike_parking_ratio['ratio difference'] > 0

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Business-with-bike-parking ratio difference between each city and average among all businesses',
          fontdict = {'fontsize' : 16})
#plt.xticks(rotation=30)

sns.barplot(y = 'ratio difference',
            x = 'city',
            hue = 'positive',
            data = bike_parking_ratio)

plt.legend('',frameon=False)
plt.xlabel('')
plt.show()

In the top 10 cities with most businesses from this dataset, all three Canadian cities--Toronto, Montreal, and Calgary--have a higher ratio of business with bike parking than the average. On the contrary, the only city in the US that does relatively well in terms of bike parking is Pittsburge, and while Las Vegas and Phoenix are the top 2 US cities in terms of number of businesses, they perform the worst to provide bike parking among the 10 cities. This results aligns with our general impression of the United States as "a nation on wheels". According to a list of 20 most bike friendly cities in the world in 2019 from WIRED (https://www.wired.com/story/most-bike-friendly-cities-2019-copenhagenize-design-index/), the only North America cities on the list are Vancouver and Montreal.

Then I look at the business-with-bike-parking ratio by categories. I considered the top 10 popular categories, and businesses containing 'bike' in their categories. 

In [ ]:
# select 10 of the most popular categories
top_categories = popular.head(10)
top_categories = top_categories['categories'].to_list()

# add category 'bike'
top_categories.append('bike')

In [ ]:
category_ratio = []

# calculate ratio by category
for category in top_categories:
    category_business = business[business['categories'].str.contains(category, case=False, na=False)]
    with_bike = category_business[category_business['attributes.BikeParking'] == 'True']
    curr_ratio = len(with_bike) / len(category_business)
    category_ratio.append(curr_ratio)

# create dataframe    
bike_category_ratio = pd.DataFrame({'category':top_categories,
                     'ratio': category_ratio}).sort_values('ratio', ascending=False)
bike_category_ratio['positive'] = bike_category_ratio['ratio'] > len(bike_parking) / len(business)

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Ratio of businesses with bike parking by category', fontdict = {'fontsize' : 16})

sns.barplot(x = 'ratio',
            y = 'category',
            color = 'm',
            data = bike_category_ratio)

plt.axvline(len(bike_parking) / len(business), linewidth=1, color='grey', linestyle='--', label = "average ratio")
plt.legend('',frameon=False)
plt.ylabel('')
plt.xlabel('ratio of businesses with bike parking')
plt.show()

Most bike-related businesses provide bike parking, as expected. In contrast, very few automotive businesses provide bike parking. It is interesting that 'Food' has a higher ratio of bike parking than 'Restaurants'. To further investigate this phenominon, I selected all the businesses with categories containing 'Food' but not 'Restaurants'. There are 15212 such businesses, and 8818 of them provide bike parking. The bike-parking ratio of 'Food-but-not-Restaurant' is 58.0%, which is relatively high among all categories, and higher than both 'Food' and 'Restaurant'.

I'm interested in the difference between 'Food' and 'Restaurant'. To investigate what causes the higher ratio of 'Food' than 'Restaurants', I selected businesses with only 'Food' in their categories but not 'Restaurants', and plot the most popular subcategories with a bar chart. There are 8812 businesses with categories 'Food' but not 'Restaurants' with bike parking.

In [ ]:
food_business = business[business['categories'].str.contains('food', case=False, na=False)]
restaurant_business = food_business[food_business['categories'].str.contains('Restaurant', case=False, na=False)]\
                    .index

# select food-but-not-restaurant businesses 
food_business.drop(restaurant_business, inplace=True)

In [ ]:
# select with bike parking businesses
food_only = food_business[food_business['attributes.BikeParking']=='True']

In [ ]:
food_only_category = explode_categories(food_only, 11)
# exclude the first row of 'food'
food_only_category = food_only_category[1:]

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Top categories of food-but-not-restaurant businesses with bike parking',fontdict = {'fontsize' : 16})

sns.barplot(x = 'count',
            y = 'categories',
            color = 'g',
            data = food_only_category)

plt.ylabel('')
plt.xlabel('Number of businesses')
plt.show()

We can see that most of them are related to drinks, snacks, or shopping, and people ususally spend less time in these businesses rather than actual restaurants for meals. Businesses such as coffee/tea shops or grocery stores might have a larger need to support biking, as more people would ride a bike for a short travel from home to these stores and go back.

#### Limitations
1. There are four distinct values in the column of bike parking ('True', 'False', 'None', nan). It is ambiguous what 'None' stands for, and I can't find an explanation in the documentation from Yelp (https://www.yelp.com/dataset/documentation/main). I intrepret 'None' the same as nan, but it could mean that 'There is no bike parking', i.e. same to 'False'.  

### 2.4 Relationship between number of reviews and ratings

There are a few businesses with a large number of reviews. I excluded the outliers using the method of interquantile range, and visualize the distribution of review counts by rating.

In [ ]:
# drop out the outliers
def remove(df):
    q1 = df['review_count'].quantile(0.25)
    q3 = df['review_count'].quantile(0.75)
    IQR = q3 - q1
    return df[(df['review_count'] < (q3 + 1.5 * IQR)) & (df['review_count'] > (q1 - 1.5 * IQR))]

rating_review = remove(rating_review)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y="review_count", x="stars", data=rating_review, palette='pastel')
plt.title("Distribution of review counts by rating",fontdict = {'fontsize' : 16})
plt.ylabel('review count')
plt.xlabel('rating')
plt.show()

Generally, businesses do not receive many reviews, as the median review count for all ratings are below 10. We can see that businesses with an average rating of 1 star and 5 stars have fewer reviews than businesses with other ratings. This is probably because that it is rare for a lot of customers to have the same extreme review on businesses, so that an extremely low or high rating is likely to be caused by a low number of reviews.

#### Limitations
1. We lose some of the valid information by removing businesses with a lot of reviews. In future analysis, we can investigate if there is any relationship between review count and ratings for businesses that receive a lot of reviews.
2. The relationship between number of reviews and ratings may vary among different categories. We could compare the relationship is different between Restaurants, Shopping, Health-related businesses, etc.

## Part 3. A Closer Look on Businesses in the GTA

I selected businesses in the Greater Toronto Area (GTA) according to the list 'Municipalities in Greater Toronto Area and related CMAs' from WikiPedia. There are 25 subdivisions in the GTA, and if the city name matches with any of the subdivisions, I would include the business in this part of analysis.

In [ ]:
url = "https://en.wikipedia.org/wiki/Greater_Toronto_Area"
df = pd.read_html(url, header=0)
gta = df[1]
# remove the last four rows (doesn't belong to GTA)
gta = gta[:-4]
gta = gta['Census subdivision'].tolist()

In [ ]:
# select businesses with city name the same as one of the subdivisions
gta_business = business.loc[business['city'].isin(gta)]

In [ ]:
# gta_business.to_csv("gta_business.csv", index=False)

In [ ]:
# gta_business = pd.read_csv('gta_business.csv')

3.1. What are the most frequent business categories? How do they compare against the trends listed in Part 2?

3.2. What are the top franchises in the city?

3.3. Does business location play an important role in reviews?

3.4. Is it true that for every Tim Hortons in the GTA there is a Starbucks nearby? Calculate distances between establishments of the two groups and assess distance patterns. Plot the two types of establishments on a map.

### 3.1 The most frequent business categories in the GTA

In [ ]:
popular_gta = explode_categories(gta_business, 15)

# select open businesses only
gta_open = gta_business[gta_business['is_open']==1]
popular_gta_open = explode_categories(gta_open, 15)

In [ ]:
plot_stacked_h(popular_gta, popular_gta_open, 
               'b', 
               "Number of total and open businesses in the top 15 most popular categories in GTA")

Again, I only considered open businesses for this part of analysis. Comparing with the top categories for all businesses, we can see that Restaurants is still by far the most popular category, but the ranking of Food rises by one in the GTA, swapping place with Shopping. While Home Services is a frequent category for all businesses, it is no longer on the top 15 list for GTA. The rankings of Beauty & Spas, Nightlife and Bars all increase for GTA businesses compared with all businesses. It may indicate that there is a larger demand for social life and entertainment in GTA than the average level in this dataset, and this result may be caused by the identity of Toronto as a metropolitan city. Compared with remote towns or rural areas in the dataset, it is reasonable that such difference arises in the Greater Toronto Area.

Next, I did the same analysis on subcategories of food-related businesses.

In [ ]:
gta_food = gta_business[gta_business['categories'].str.contains(
                         'Restaurants|Food', case=False, na=False)]

popular_gta_food = explode_categories(gta_food, 17)
# remove rows of 'Restaurants' and 'food'
popular_gta_food = popular_gta_food[2:]

# select open restaurants only
gta_food_open = gta_food[gta_food['is_open']==1]
popular_gta_food_open = explode_categories(gta_food_open, 17)
popular_gta_food_open = popular_gta_food_open[2:]

In [ ]:
plot_stacked_h(popular_gta_food, popular_gta_food_open, 
               'm', 
               "Number of total and open businesses in the top 15 most popular food-related categories in GTA")

American(Traditional) and American(New) is not found on the list for GTA, but Canadian(New) appears, as expected. However, it is interesting that the ranking of Chinese food rises from 14th in all businesses to 5th in GTA businesses, which might to some extent reflect the demographic difference between GTA and the average in North America.

#### Limitations
1. We might be missing some businesses that are actually in GTA using selection by city name. An alternative is to select by postal code, as postal code starting with 'M' and 'L' are considered to be in GTA. However, using that method, some businesses that are not in GTA might be included.

### 3.2 The Top Franchises in the GTA

I noticed many spelling mistakes and abbrevations for business names. So I find the top 20 franchises first, and compare all of the GTA business names with the top 20 franchises using fuzzywuzzy. In particular, Tim Hortons has many different variations to be corrected.

In [ ]:
# find the top 20 franchises
gta_franchise_count = gta_business[["name", "business_id"]].groupby("name").count().reset_index()
gta_franchise_count.columns = ["name", "count"]
gta_head = gta_franchise_count.sort_values("count", ascending=False).head(20)

In [ ]:
for idx, miss in gta_business.iterrows():
    max_score = 0
    max_index = 0
    
    # Compare the similarity of the incorrect city name to each correct city name 
    for i, exist in gta_head.iterrows():
        score = fuzz.ratio(miss['name'], exist['name'])
        
        # iterate to find max score
        if score > 80 and score > max_score:
            max_score = score
            max_index = i
            gta_business.at[idx, 'name'] = exist['name'] 

In [ ]:
# check if all Tim Hortons have been fixed
tim = gta_business[gta_business['name'].str.contains('Tim Horton', case=False, na=False)]
# see unique names that could be Tim Hortons
# tim['name'].unique()

In [ ]:
# replace names starting with 'Tim Horton' with 'Tim Hortons'
gta_business = gta_business.replace({'name': '^Tim Horton.*$'}, {'name': 'Tim Hortons'}, regex=True)

In [ ]:
# calculate the total number again
gta_franchise_count = gta_business[["name", "business_id"]].groupby("name").count().reset_index()
gta_franchise_count.columns = ["name", "count"]
gta_head = gta_franchise_count.sort_values("count", ascending=False).head(15)

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Top franchises in GTA',fontdict = {'fontsize' : 16})

sns.barplot(x = 'count',
            y = 'name',
            color = 'rosybrown',
            data = gta_head)

plt.ylabel('')
plt.xlabel('Number of businesses')
plt.show()

Tim Hortons, Starbucks are the top 2 franchises in GTA, with a similar number of businesses (239 for Tim Hortons and 237 for Starbucks) that are well above others. Most of the top franchises are restaurant chains, with Shoppers Drug Mart, GoodLife Fitness, and LCBO as the exceptions.

#### Limitations
1. There are still some business names that are not corrected even with fuzzywuzzy. But since we only focus on the top franchises, and the following analysis mainly involves Tim Hortons and Starbucks, the errors will not have much effect on my results.

### 3.3 Does business location play an important role in reviews?

For this part of analysis, I only considered businesses that are open, and there are 22647 open businesses in the GTA. First, I plotted all open businesses in GTA and color-coded the points with number of stars, but the map is not very informative, as there are too many businesses and the points are stacked over each other. Therefore, it is impossible to see any relationship between ratings and locations.

Then I selected the top 500 businesses with best ratings, and the top 500 businesses with most reviews, and plotted two maps separately.

In [ ]:
open_business = gta_business[gta_business['is_open']==1]

In [ ]:
fig = px.scatter_mapbox(open_business, lat="latitude", lon="longitude", hover_name="name", hover_data=["stars"],
                        color='stars',color_continuous_scale=px.colors.diverging.Portland,
                        opacity=0.1, zoom=10, height=350)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<center>
    <img src="map2.png"></img>
</center>

In [ ]:
# select the top 500 businesses with best ratings
top = open_business.sort_values('stars', ascending=False)
top = top.head(500)

In [ ]:
fig = px.scatter_mapbox(top, lat="latitude", lon="longitude", hover_name="name", hover_data=["stars"],
                        color='review_count', 
                        color_continuous_scale=px.colors.sequential.Oryel,
                        opacity=1, zoom=10, height=350)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<center>
    <img src="map3.png"></img>
</center>

There are still no obvious patterns for businesses with good ratings, and they are scattered everywhere. Most of them do not have a high number of reviews either, with one exception--a coffee bar with 5 stars have more than 200 reviews, and we will investigate this case in Part 4 to see if this coffee bar is buying fake reviews, or it is actually wonderful.

In [ ]:
# select the top 500 businesses with most reviews
busy = open_business.sort_values('review_count', ascending=False)
busy = busy.head(500)

In [ ]:
fig = px.scatter_mapbox(busy, lat="latitude", lon="longitude", hover_name="name", hover_data=["stars"],
                        color='stars',
                        color_continuous_scale=px.colors.diverging.Portland,
                        opacity=0.9, zoom=10, height=350)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<center>
    <img src="map4.png"></img>
</center>

For businesses with most reviews, we can see that the clustering in downtown Toronto is heavier than businesses with best ratings. Outside of downtown, there are two obvious patterns: one going south and north along Yonge Street, and the other going west and east in Markham near Highway 7. Since both of them are main road with busy traffic, it is possible that number of reviews is related to number of customers and sales.

#### Limitations
1. We don't have the data of sales or profits, so we cannot verify the possible relationship between review counts and popularity of a business.
2. In further analysis, we can zoom in on specific districts. For example, downtown Toronto, which has a lot of businesses. A lot of information is missed by looking at the map of the whole area, because the points are too clustered.

### 3.4 Is it true that for every Tim Hortons in the GTA there is a Starbucks nearby?

First, I plotted all of the Tim hortons (red dots) and Starbucks (green dots) on a map. We can see that indeed the distance between red dots and green dots are very close, especially in Downtown. However, moving further away from downtown area, there seems to be more Tim Hortons than Starbucks.

Next, I calculate the distance between every Tim Hortons and its nearest Starbucks.

In [ ]:
tim_hortons_b = gta_business[gta_business['name'] == 'Tim Hortons']
starbucks_b = gta_business[gta_business['name'] == 'Starbucks']

# only select necessary columns
tim_hortons_b = tim_hortons_b[['latitude','longitude','business_id']]
starbucks_b = starbucks_b[['latitude','longitude','business_id']]

In [ ]:
fig = go.Figure()

# plot Tim Hortons
fig.add_trace(go.Scattermapbox(
        lat=tim_hortons_b['latitude'],
        lon=tim_hortons_b['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            color='rgb(220, 15, 45)',
            opacity=0.7
        ),
        hoverinfo='none'
    ))

# plot Starbucks
fig.add_trace(go.Scattermapbox(
        lat=starbucks_b['latitude'],
        lon=starbucks_b['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            color='rgb(0,112,74)',
            opacity=0.7
        ),
        hoverinfo='none'
    ))

fig.update_layout(
    title='Locations of Tim Hortons and Starbucks in the GTA',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        bearing=0,
        pitch=0,
        zoom=3,
        style='carto-positron'
    ),
)

fig.show()

<center>
    <img src="map5.png"></img>
</center>

In [ ]:
def calc_distance(lat_from, lon_from, lat_to, lon_to):
    """
    Calculate distance between two coordinates represented by 
    latitude and longitude, and return distance in km.
    """
    coords_1 = (lat_from, lon_from)
    coords_2 = (lat_to, lon_to)
    distance = geopy.distance.distance(coords_1, coords_2).km
    
    return distance

In [ ]:
distance_between = []
for idx, store in tim_hortons_b.iterrows():
    min_distance = 100
    # get coordinate of Tim Hortons
    tim_lat = tim_hortons_b.at[idx, 'latitude']
    tim_lon = tim_hortons_b.at[idx, 'longitude']
    
    # iterate through every Starbucks
    for i, starbucks in starbucks_b.iterrows():
        star_lat = starbucks_b.at[i, 'latitude']
        star_lon = starbucks_b.at[i, 'longitude']
        # calculate distance between Tim Hortons and this Starbucks
        curr = calc_distance(tim_lat, tim_lon, star_lat, star_lon)
        
        # iterate to get min
        if curr < min_distance:
            min_distance = curr
    
    distance_between.append(min_distance)

In [ ]:
distance_between = pd.DataFrame(distance_between).rename(columns={0: "distance"})

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y="distance", data=distance_between)
plt.title("Distribution of distance between Tim Hortons and its nearest Starbucks", fontdict = {'fontsize' : 16})
plt.ylabel('Distance', fontdict = {'fontsize' : 14})
plt.xlabel('')
plt.show()

From the box plot, we can see that the median distance between Tim Hortons and Starbucks is below 1 km. However, there are some outliers and the max distance is 7.92 km, which is likely to be found in rural areas. It is interesting that the min value is zero, as a Tim Hortons and Starbucks may be in the same building (e.g. Medical Sciences Building at the University of Toronto campus).

#### Limitations
1. We could calculate the distance of Starbucks to its nearest Tim Hortons and make a comparison.
2. We could investigate by districts, such as separating urban areas and rural areas.

## Part 4. An Analysis on User Reviews

Since the review file is very large, I loaded it by chunks and merged with GTA businesses data.

In [ ]:
# Set up local path
review_json_path = 'review.json'

In [ ]:
# Set chunk size (smaller if dataset is smaller)
# 2019 Yelp review.json has more than 6 million reviews(rows)
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size)

In [ ]:
# only select relavant columns
gta_reviews = gta_business[['business_id', 'name', 'stars', 'review_count']]

In [ ]:
# There are multiple chunks to be read
chunk_list = []
for chunk in review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(gta_reviews, chunk, on='business_id', how='inner')
    # Show feedback on progress
    # print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
    
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
# print(df.shape)
# df.sample(3)

In [ ]:
# save csv for later use
# df.to_csv("gta_reviews.csv", index=False)

In [ ]:
# df = pd.read_csv('gta_reviews.csv')

### 4.1 Is there a small group of users responsible for most reviews?

There are 709928 reviews for all GTA businesses, and these reviews are written by 141973 users. 

In [ ]:
user_review_count = df.groupby(['user_id'])['user_id'].count().reset_index(name='count')\
                    .sort_values('count', ascending=False)

In [ ]:
plt.figure(figsize=(8,6))

# Draw the density plot
graph = sns.distplot(user_review_count['count'], hist = False, 
                     kde = True, kde_kws = {'shade': True,'linewidth': 3})
    
# Plot formatting
plt.title('Distribution of number of reviews written by each user for GTA businesses')
plt.xlabel('Number of reviews written')
plt.ylabel('Density')

plt.show()

From the plot of distribution, we can see that most users only wrote a few review, while there is one user who wrote over 3,000 reviews. I divided the users into four groups by number of reviews written(1, 2-10, 11-100, >100). 

In [ ]:
# group user by number of reviews written
one_review_user = user_review_count[user_review_count['count'] == 1]
ten = user_review_count.loc[(user_review_count['count'] > 1) & (user_review_count['count'] <= 10)]
hundred = user_review_count.loc[(user_review_count['count'] > 10) & (user_review_count['count'] <= 100)]
more_than_hundred = user_review_count[user_review_count['count'] > 100]

num_review = pd.DataFrame({'user group':['1 review', '2-10 reviews', '11-100', '> 100'],
                     'count': [len(one_review_user), len(ten),len(hundred), len(more_than_hundred)]})

In [ ]:
plt.figure(figsize=(8,6))

colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
sizes = num_review['count'].to_list()

plt.pie(sizes, labels=num_review['user group'], colors=colors, autopct='%1.1f%%', shadow=False, startangle=90)
plt.axis('equal')
plt.tight_layout()
plt.suptitle('Proportion of number of users in each group by number of reviews', size=16, y=1.05)
plt.show()

More than half of the users only wrote 1 review, and more than 90% of the users wrote no more than 10 reviews. On the contrary, 0.5% users wrote over 100 reviews, and they are responsible for most reviews.

### 4.2 Do Yelp reviewers use similar language in their reviews of GTA's Tim Horton's and Starbucks?

There are 1518 reviews written for Tim Hortons, and 2512 reviews for Starbucks. I tokenized the reviews and plotted a wordcloud for Tim Hortons and Starbucks separately.

In [ ]:
tim_hortons = df[df['name'] == 'Tim Hortons']
starbucks = df[df['name'] == 'Starbucks']

In [ ]:
# join all reviews to one string
tim_text = " ".join(review for review in tim_hortons.text)
star_text = " ".join(review for review in starbucks.text)

In [ ]:
def tokenize_review(string):
    '''
    tokenize reviews and return a sentence for plotting wordcloud.
    '''
    from nltk.corpus import stopwords
    
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(string) 
  
    filtered = [w for w in word_tokens if not w in stop_words] 
  
    filtered = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered.append(w) 
        
    sentence = " ".join(word for word in filtered)
    return sentence

In [ ]:
tim_sentence = tokenize_review(tim_text)
star_sentence = tokenize_review(star_text)

In [ ]:
stopwords = set(STOPWORDS)
# remove words that are not informative
stopwords.update(["Tim Hortons", "Hortons","Tim","Tim ","re","something","going","think",
                  "know","came","try","see","work","said","give","come","went","ll","ordered", 
                  "Horton", "make", "flavors", "n't", "got", "usually", "really", "one","back",
                  "thing", "seem", "alway", "always", "ve", "go", "much", "around", "even","way"])

plt.figure(figsize=(8,6))
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(tim_sentence)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most common words in reviews for Tim Hortons', fontdict = {'fontsize' : 18})
#plt.rcParams.update({'font.size': 20})
plt.axis("off")
plt.show()

In [ ]:
stopwords = set(STOPWORDS)

stopwords.update(["Starbucks","re","something","going","think",
                  "know","came","try","see","work","said","give","come","went","ll","ordered", 
                  "Horton", "make", "flavors", "n't", "got", "usually", "really", "one","back",
                  "thing", "seem", "alway", "always", "ve", "go", "much", "around", "even","way"])

plt.figure(figsize=(8,6))
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(star_sentence)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most common words in reviews for Starbucks', fontdict = {'fontsize' : 18})
plt.axis("off")
plt.show()

'Location' is mentioned very frequently for both Tim Hortons and Starbucks. For Tim Hortons, we notice more specialty food (Timbits, Timmie, etc.), and for Starbucks, customers seem to comment more about services (staff, barista, etc.)

Then I looked at users who wrote reviews for both Tim Hortons and Starbucks by matching user id, and there are 792 such users.

In [ ]:
# find users who wrote reviews for both
tim_users = tim_hortons['user_id'].to_list()
both = starbucks.loc[starbucks['user_id'].isin(tim_users)]

both_user = both['user_id'].to_list()
tim_starbucks = both.append(tim_hortons.loc[tim_hortons['user_id'].isin(both_user)])

In [ ]:
tim_text_both = " ".join(review for review in tim_starbucks[tim_starbucks['name']=='Tim Hortons'].text)

tim_sentence_both = tokenize_review(tim_text_both)

In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["Tim Hortons", "Hortons","Tim","Tim ","re","something","going","think",
                  "know","came","try","see","work","said","give","come","went","ll","ordered", 
                  "Horton", "make", "flavors", "n't", "got", "usually", "really", "one","back",
                  "thing", "seem", "alway", "always", "ve", "go", "much", "around", "even","way"])

plt.figure(figsize=(8,6))
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(tim_sentence_both)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most common words in reviews for Tim Hortons by users who wrote review for both', 
          fontdict = {'fontsize' : 18})
#plt.rcParams.update({'font.size': 20})
plt.axis("off")
plt.show()

In [ ]:
star_text_both = " ".join(review for review in tim_starbucks[tim_starbucks['name']=='Starbucks'].text)

star_sentence_both = tokenize_review(star_text_both)

In [ ]:
stopwords = set(STOPWORDS)

stopwords.update(["Starbucks","re","something","going","think",
                  "know","came","try","see","work","said","give","come","went","ll","ordered", 
                  "Horton", "make", "flavors", "n't", "got", "usually", "really", "one","back",
                  "thing", "seem", "alway", "always", "ve", "go", "much", "around", "even","way"])

plt.figure(figsize=(8,6))
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(star_sentence_both)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most common words in reviews for Starbucks by users who wrote review for both', 
          fontdict = {'fontsize' : 18})
plt.axis("off")
plt.show()

The wordclouds are similar. So I chose to compare the average word counts of reviews with word counts of reviews written by users who wrote review for both.

In [ ]:
def calc_review_len(store):
    """
    Calculate average word counts of reviews for a business.
    """
    total = 0
    count = 0

    for idx, review in store.iterrows():
        sentence = store.at[idx, 'text']
        # separate words
        words = sentence.split(' ')
        curr = len(words)
        total += curr
        count += 1

    return total/count        

In [ ]:
print(calc_review_len(tim_hortons), calc_review_len(starbucks), 
      calc_review_len(tim_starbucks[tim_starbucks['name']=='Tim Hortons']),
      calc_review_len(tim_starbucks[tim_starbucks['name']=='Starbucks']))

The average word count of all reviews for Tim Hortons is 87.3, and for Starbucks is 88.7. In comparison, users who wrote reviews for both seem to write longer reviews, as the average word count for Tim Hortons is 93.0, and for Starbucks is 96.4.

#### Limitations
1. It is questionable whether we should remove some words in the wordcloud. For example, for 'make', we don't know if it mean 'I make the decision to go' (irrelavant) or 'The baristars make drinks quickly'(relavant).

### 4.3 Can we automatically detect "paid reviewers" (i.e. people who are paid to write positive reviews)?

From the previous analysis, I noticed one shop as an anomaly with 5 stars has more than 200 reviews--Baretto Caffe, and I want to check if this shop is buying fake reviews. First, I calculated the average length of review, which is 92, similar to the length of reviews for Tim Hortons and Starbucks. This suggests no evidence for fake reviews. Then I read the actual reviews, and they look natural too, so I consider those as real reviews.

In [ ]:
# get the name 'baretto caffe' from label on map previously, and find its reviews
baretto_caffe = df[df['name'] == 'Baretto Caffe']

baretto_ave = calc_review_len(baretto_caffe)

I think of several possible ways to detect paid reviews, but each has its flaws, so I was not able to implement those:
1. A person who rated a lot of 5-stars and 5-stars only may be paid reviewers. But it is possible that they are nice and only review businesses they like.
2. A person who only one 5 star may be paid to write the review, as it is not something they would normally do. However, more than 50% of users only wrote 1 review and it is hard to pick paid reviewers from real reviewers.

There are many caveats in determining what counts as 'paid reviewers' too. For example, a business could let their employees to write reviews, or encourage customers to write reviews by giving coupons, and it is ambigious whether such cases should be considered as 'fake reviews'. 

Therefore, without the labels for real/fake reviews, we cannot automatically detect them.

## Conclusion

While this report mainly explores business and review data, a lot of them are descriptive, and we would need extra source within the Yelp dataset(e.g. checkin, user) as well as external dataset to find more useful conclusions for business owners as well as customers.